In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 환경변수 및 패키지

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_ollama import ChatOllama
import os
import re
import ollama

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage

from typing import List, Optional, Dict, Any
from __future__ import annotations
from collections import defaultdict

load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'
OLLAMA_EMBEDDING_MODEL = 'bge-m3'

PINECONE_INDEX_NAME_LAW = 'law-index-final'
PINECONE_INDEX_NAME_RULE = 'rule-index-final'
# PINECONE_INDEX_NAME_CASE = 'case-index-final'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

# 1. 함수

In [4]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전
    Returns:
        dict: {카테고리명: {키워드: 가중치}}
    """
    return {
        # 1. 보증금 보호 (가장 치명적인 리스크)
        '보증금_대항력': {
            '보증금': 3, '전세금': 2, '보증금반환': 3, '반환청구': 2,
            '대항력': 3, '전입신고': 2, '주민등록': 2, '점유': 2, '인도': 2,
            '우선변제권': 3, '최우선변제': 3, '소액임차인': 3, '배당': 2, '경매': 2, '공매': 2,
            '확정일자': 3, '확정일자부': 2,'임차권등기': 3, '임차권등기명령': 3,
            '반환보증': 2, '보증보험': 2, 'HUG': 1, '보증보험': 2,
            '대항요건': 3,'제삼자': 2,'효력': 1,'효력발생': 2, '입주': 1,
            '거주': 1,'주택인도': 2, '임대차계약증서': 2,'환가대금': 2,
            '후순위권리자': 2,'변제': 1,'배당요구': 2,'임차권등기명령': 3,'임대차등기': 2,
            '지방법원': 2,'등기촉탁': 1,'등기신청': 1,
        },

        # 2. 계약 갱신 및 거절 (가장 빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신요구권': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '재계약': 2, '존속기간': 2, '2년': 1,
            '실거주': 3, '거절사유': 2, '손해배상': 2,
            '연체': 2, '2기': 2, '의무위반': 2,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1, '계약갱신청구권': 3,
            '갱신거절통지': 3,'계약조건변경': 2, '계약만료': 2,'직계존속': 2,
            '직계비속': 2,'직계존비속': 2, '중개보수': 1,
        },

        # 3. 계약 해지 및 종료 (탈출 전략)
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },

        # 4. 차임(월세) 및 보증금 증감
        '임대료_증감': {
            '월세전환': 3, '반전세': 2, '차임': 3, '월세': 2, '증액': 3, 
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2, '감액': 2, '인상': 2, '1년': 2, '1년 이내': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1, '전환되는 금액': 2,
            '초과차임': 2, '반환': 2, '증액청구': 3,'증액비율': 2, '차임액': 2,
            '보증금증액': 2, '차임연체': 2, '2기차임': 2,'연체': 1,
        },

        # 5. 유지보수 및 원상회복 (민법 영역)
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2, '유지': 2, '하자': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },

        # 6. 전세사기 및 권리 리스크 (독소조항 탐지용)
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3, '신탁부동산': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, 
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2, '등기부등본': 2,
            '말소기준권리': 3, '채권최고액': 2, '가등기': 2, '전세권': 2, '전세권등기': 2,
            '소유권이전등기': 2,'말소': 2, '배당요구': 2,'건축물대장': 2, '보존등기': 2, '이행강제금': 1,
        },

        # 7. 행정 절차 (Rules Index 연결)
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1, '등기부등본': 3,
            '건축물대장': 3,'중개대상물확인설명서': 2,'공인중개사': 1,'중개수수료': 2, '중개보수': 2,
        },

        # 8. 소송 및 분쟁 해결 (최후 수단)
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3, '조정': 3, '조정부': 2,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2, '조정신청': 3, '신청': 2, '각하': 2,
            '조정절차': 3, '송달': 2, '서면': 2, '구두': 2,
            '처리기간': 2, '60일': 2, '30일': 1,
            '심사관': 1, '조사관': 1, '사무국': 1,
            '조정신청': 3,'조정안': 2,'조정서': 2, '조정성립': 2,'인도명령': 2,'명도소송': 2,
            
            # 소송/집행 트리거
            '지급명령': 3, '소송': 3, '판결': 2, '집행권원': 2,
            '경매': 3, '공매': 2, '내용증명': 2, '손해배상': 3, '지연이자': 2,

            # 시행령(원문 표기): 조정 대상 범위 + 신청 절차 상세
            '제22조(조정위원회의 심의ㆍ조정 사항)': 3,
            '제29조(조정부의 구성 및 운영)': 1,
            '제30조(조정의 신청)': 2,
            '제31조(조정신청인에게 안내하여야 할 사항)': 1,
            '제32조(조정서류의 송달 등)': 1,
        },
        
        # 9. 임차권 승계 및 가족 
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        },
        
        # 10. 전대/양도
        '전대_양도': {
            '전대': 3, '전차': 2, '전대차': 2, '임대인의 동의': 3,
            '양도': 2, '임차권 양도': 2
        },

        # 0. 적용 범위/예외(필요 시 사용)
        '적용범위_예외': {
            '적용범위': 3, '주거용': 2, '주택 일부': 2, '주거 외 목적': 2,
            '일시사용': 3, '일시사용하기 위한 임대차': 3,
            '미등기': 3, '미등기 전세': 3, '전세계약': 2
        },
    }


def categorize_content(content, top_k=None):
    '''
    내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters : 
        - content : 분류할 텍스트 내용
        - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
        
    Returns :
        - 카테고리 리스트(점수 높은 순)
    '''
    category_keywords = get_law_category()
    category_scores = {}
    
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0 :
            category_scores[category] = score
    
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x : x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    # 매칭되는 카테고리가 없으면 '기타' 반환
    if not all_categories:
        all_categories = ['기타']
        
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    return all_categories


# =====================
# 참조조항을 추출하는 함수
# =====================

_ARTICLE_RE = re.compile(r"(?:제\s*)?(\d+)\s*조(?:\s*의\s*(\d+))?")

def _law_name_from_src_title(src_title: str) -> str:
    if not src_title:
        return "알수없음"
    s = str(src_title).strip()
    if not s:
        return "알수없음"
    # 첫 '(' 이전을 법령명으로 사용
    name = s.split("(", 1)[0].strip()
    return name if name else "알수없음"

def _split_article_field(article_field) -> list[str]:
    if not article_field or article_field == "조항없음":
        return []
    s = str(article_field)
    s = s.replace("「", "").replace("」", "").strip()
    if not s:
        return []
    # ",", ";", 줄바꿈, "·" 등 다양한 구분자 대응
    tokens = re.split(r"[,\n;·]+", s)
    return [t.strip() for t in tokens if t and t.strip()]

def _parse_article_token(token: str):
    if not token:
        return None
    token = re.sub(r"\s+", "", token)
    m = _ARTICLE_RE.search(token)
    if not m:
        return None
    main = int(m.group(1))
    sub = int(m.group(2)) if m.group(2) else 0
    return (main, sub)

def _format_article_key(key):
    main, sub = key
    return f"제{main}조" if sub == 0 else f"제{main}조의{sub}"

def extract_articles_from_docs(documents):
    """
    검색된 문서들(documents)에서 article 정보를 추출하여
    '법령명: 제x조,제y조 | 법령명2: ...' 형태로 반환
    """
    # law_name -> set( (main,sub) )
    buckets = defaultdict(set)
    # law_name -> best priority(작을수록 우선)
    law_priority = {}

    for doc in documents:
        meta = doc.metadata or {}

        law = _law_name_from_src_title(meta.get("src_title") or meta.get("law_name") or meta.get("source"))
        pr = meta.get("priority", None)
        try:
            pr_int = int(pr) if pr is not None else None
        except Exception:
            pr_int = None

        if pr_int is not None:
            prev = law_priority.get(law)
            if prev is None or pr_int < prev:
                law_priority[law] = pr_int

        for tok in _split_article_field(meta.get("article", "조항없음")):
            key = _parse_article_token(tok)
            if key is not None:
                buckets[law].add(key)

    if not buckets:
        return "(참조 조항 없음)"

    # 법령 블록 정렬: priority(있으면) -> 법령명
    def law_sort_key(law_name: str):
        p = law_priority.get(law_name)
        return (p if p is not None else 9999, law_name)

    parts = []
    for law in sorted(buckets.keys(), key=law_sort_key):
        keys = sorted(buckets[law])
        if not keys:
            continue
        parts.append(f"{law} " + ",".join(_format_article_key(k) for k in keys))

    return " | ".join(parts) if parts else "(참조 조항 없음)"



# ====================================
# 검색된 document를 텍스트로 변환하는 함수
# ====================================

def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])


# ===========================
# title+text 기반의 rerank함수
# ===========================

import cohere
import numpy as np

def rerank_by_title(query: str, documents: list, top_k: int = 4):
    '''
    cohere의 rerank API를 사용한 문서 재정렬
    '''
    # cohere client 초기화
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    # 0건이면 rerank 호출하지 말고 그대로 반환
    if not documents:
        return []

    # top_k도 0이면 그대로
    if not top_k or top_k <= 0:
        return documents

    docs_text = [
        (doc.metadata.get("src_title", "") or "") + " " + (doc.page_content or "")
        for doc in documents
    ]

    # documents가 비어있지 않아도, docs_text가 전부 빈문자면 Cohere가 싫어할 수 있어서 방어
    if not any(t.strip() for t in docs_text):
        return documents[:top_k]

    results = co.rerank(
        model="rerank-multilingual-v3.0",
        query=query,
        documents=docs_text,
        top_n=min(top_k, len(docs_text)),
    )

    idxs = [r.index for r in results.results]
    return [documents[i] for i in idxs]


# ==========================================
# 검색된 document를 LLM용 context로 변환 (개선)
# ==========================================

def format_documents_with_meta(documents, max_len: int = 500):
    """
    프롬프트.txt 구조 반영:
    - 법령명 / 조문 / priority 명시
    - LLM이 법적 근거를 명확히 인식하도록 설계
    """
    context_parts = []

    for i, doc in enumerate(documents, 1):
        meta = doc.metadata or {}

        law_name = meta.get("law_name") or meta.get("src_title") or meta.get("source", "알 수 없는 법령")
        article = meta.get("article", "")
        priority = meta.get("priority", "?")

        content = doc.page_content.strip()
        if len(content) > max_len:
            content = content[:max_len].rstrip() + "…"

        context_parts.append(
            f"""[문서 {i}]
- 법령명: {law_name}
- 조문: {article}
- 우선순위(priority): {priority}
- 내용:
{content}
"""
        )

    return "\n\n".join(context_parts)

## 우선순위

In [5]:
import re

def sanitize_for_upstage(text: str, max_len: int = 500) -> str:
    """Upstage embedding input 안전 정제: str 보장, 공백/개행 정리, 길이 제한"""
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    # 이상한 프롬프트 잔재 제거(가끔 '질문 :' 같은 prefix가 섞임)
    text = re.sub(r"^\s*(질문\s*:|question\s*:|q\s*:)\s*", "", text, flags=re.IGNORECASE)

    # 개행/탭 과다를 단일 공백으로
    text = re.sub(r"\s+", " ", text).strip()

    # 너무 길면 자르기
    if len(text) > max_len:
        text = text[:max_len].strip()

    return text

def safe_embedding_query(normalized_query: str, original_query: str) -> str:
    q = sanitize_for_upstage(normalized_query)
    if not q:  # 빈 문자열이면 원문으로 폴백
        q = sanitize_for_upstage(original_query)
    return q


def retrieve_law_docs_priority_mix(
    law_db,
    embedding,
    query_for_embed: str,
    categories: Optional[List[str]] = None,
    k_total: int = 6,
    k_each: int = 8,
    # ✅ 우선순위 정책: 1(법) 먼저, 2(시행령) 최소 확보, 그 다음 4(민법) 등
    priority_order: List[int] = [1, 2, 4, 3, 5, 6, 7],
    # ✅ 최소 할당량(시행령 1개 확보)
    min_quota: Optional[Dict[int, int]] = None,
):
    if min_quota is None:
        min_quota = {1: max(1, k_total - 1), 2: 1}  # 기본: 1에서 k_total-1, 2에서 1

    qvec = embedding.embed_query(query_for_embed)
    picked: List[Document] = []
    seen = set()

    def add_docs(docs: List[Document]):
        nonlocal picked
        for d in docs:
            key = (
                d.metadata.get("id"),
                d.metadata.get("chunk_id"),
                d.metadata.get("source"),
                d.page_content[:120],
            )
            if key in seen:
                continue
            seen.add(key)
            picked.append(d)

    def query_priority(p: int, k_take: int):
        # ✅ filter 구성 (category는 있으면 넣고, 없으면 생략)
        flt = {"priority": {"$in": [p, float(p)]}}  # 2 vs 2.0 방어
        if categories:
            flt["category"] = {"$in": categories}

        return law_db.similarity_search_by_vector(
            qvec,
            k=min(k_each, k_take) if k_take > 0 else k_each,
            filter=flt,
        )

    # 1) 최소 할당(min_quota) 먼저 채우기
    for p in priority_order:
        need = min_quota.get(p, 0)
        if need <= 0:
            continue
        docs = query_priority(p, need)
        add_docs(docs[:need])  # 필요한 만큼만
        if len(picked) >= k_total:
            return picked[:k_total]

    # 2) 남은 수를 priority_order 순서대로 채우기
    for p in priority_order:
        if len(picked) >= k_total:
            break
        remaining = k_total - len(picked)
        docs = query_priority(p, remaining)
        add_docs(docs)

    return picked[:k_total]


## 비용계산

In [6]:
def _normalize_token_usage(d: Dict[str, Any]) -> Optional[Dict[str, Optional[int]]]:
    """
    여러 provider 형태의 usage dict를 LangChain 표준 키로 정규화.
    표준: input_tokens / output_tokens / total_tokens
    (OpenAI 계열): prompt_tokens / completion_tokens / total_tokens
    """
    if not isinstance(d, dict) or not d:
        return None

    # 1) 이미 표준 키인 경우
    input_t = d.get("input_tokens")
    output_t = d.get("output_tokens")
    total_t = d.get("total_tokens")

    # 2) OpenAI 스타일 키인 경우
    if input_t is None:
        input_t = d.get("prompt_tokens")
    if output_t is None:
        output_t = d.get("completion_tokens")

    # 3) 다른 변형 키들(혹시 모를 케이스)
    if input_t is None:
        input_t = d.get("prompt") or d.get("input")
    if output_t is None:
        output_t = d.get("completion") or d.get("output")

    # total 없으면 계산
    if total_t is None and isinstance(input_t, int) and isinstance(output_t, int):
        total_t = input_t + output_t

    # 최소한 input/output 둘 중 하나라도 있어야 유효
    if input_t is None and output_t is None and total_t is None:
        return None

    return {
        "input_tokens": input_t if isinstance(input_t, int) else None,
        "output_tokens": output_t if isinstance(output_t, int) else None,
        "total_tokens": total_t if isinstance(total_t, int) else None,
    }


def _usage_metadata_to_dict(usage_meta: Any) -> Optional[Dict[str, Any]]:
    """
    usage_metadata가 dict일 수도 있고, UsageMetadata 객체(속성)일 수도 있어 방어적으로 dict로 변환.
    """
    if usage_meta is None:
        return None

    if isinstance(usage_meta, dict):
        return usage_meta

    # pydantic/dataclass/객체 형태 대비: 속성 접근
    out = {}
    for k in ("input_tokens", "output_tokens", "total_tokens"):
        if hasattr(usage_meta, k):
            out[k] = getattr(usage_meta, k)
    return out or None


def get_token_usage(llm: Any, llm_response: Any) -> Optional[Dict[str, Optional[int]]]:
    """
    Provider-agnostic 토큰 사용량 추출 (OpenAI/Upstage 등 공통).
    - 1순위: llm_response.usage_metadata  (LangChain 표준)
    - 2순위: llm_response.response_metadata["token_usage"] (OpenAI 계열 자주 존재)
    - 3순위: llm_response.response_metadata["usage"] / ["usage_metadata"] 등 변형
    - 그 외: None
    """
    # 1) LangChain 표준 usage_metadata
    usage_meta = _usage_metadata_to_dict(getattr(llm_response, "usage_metadata", None))
    normalized = _normalize_token_usage(usage_meta or {})
    if normalized:
        return normalized

    # 2) response_metadata 쪽에서 찾기
    meta = getattr(llm_response, "response_metadata", None) or {}
    if isinstance(meta, dict):
        # 가장 흔한 케이스: token_usage
        normalized = _normalize_token_usage(meta.get("token_usage") or {})
        if normalized:
            return normalized

        # provider에 따라 usage 라는 이름을 쓰는 케이스들
        normalized = _normalize_token_usage(meta.get("usage") or {})
        if normalized:
            return normalized

        normalized = _normalize_token_usage(meta.get("usage_metadata") or {})
        if normalized:
            return normalized

        # 깊게 한번 더(혹시 중첩된 경우)
        for key in ("response", "raw", "data", "result"):
            nested = meta.get(key)
            if isinstance(nested, dict):
                for k2 in ("token_usage", "usage", "usage_metadata"):
                    normalized = _normalize_token_usage(nested.get(k2) or {})
                    if normalized:
                        return normalized

    return None


OPENAI_PRICES = {
    "gpt-4o-mini": {"input": 0.15 / 1000000,"output": 0.60 / 1000000},
    "gpt-4.1-mini": { "input": 0.40 / 1000000, "output": 1.60 / 1000000},
    "gpt-5-mini": {"input": 0.25 / 1000000, "output": 2.00 / 1000000},
    "gpt-5-nano": { "input": 0.05 / 1000000, "output": 0.40 / 1000000},
    "gpt-4.1-nano": { "input": 0.10 / 1000000, "output": 0.40 / 1000000},  
}

UPSTAGE_PRICES = {
    "solar-pro2": { "input": 0.15 / 1000000, "output": 0.60 / 1000000},
}

ALL_PRICES = {**OPENAI_PRICES, **UPSTAGE_PRICES}

def calculate_llm_cost(model_name: str, token_usage: dict) -> float | None:
    """
    OpenAI + Upstage 공통 비용 계산.
    """
    if not token_usage:
        return None

    price = ALL_PRICES.get(model_name)
    if not price:
        return None

    in_tok = token_usage.get("input_tokens")
    out_tok = token_usage.get("output_tokens")
    if not isinstance(in_tok, int) or not isinstance(out_tok, int):
        return None

    return in_tok * price["input"] + out_tok * price["output"]

## RAGAS

In [7]:
import re
from dataclasses import dataclass
from typing import List, Tuple, Set, Dict, Optional

# 답변에서 조문(조/조의) 탐지
_ART_IN_ANSWER_RE = re.compile(r"(제\s*\d+\s*조(?:\s*의\s*\d+)?)")

# 문장 분리(간단): 줄바꿈/마침표/물음표/느낌표/‘다.’ 기준
_SENT_SPLIT_RE = re.compile(r"(?:(?<=[\.\!\?])\s+|(?<=다\.)\s+|\n+)")

def _norm_art(s: str) -> str:
    # "제 6 조 의 2" -> "제6조의2"
    return re.sub(r"\s+", "", s).replace("조의", "조의")

def _norm_article_token(tok: str) -> Optional[str]:
    """
    메타데이터 article 토큰을 '제N조' or '제N조의M'으로 정규화
    입력 예: "20조", "21 조", "제22조", "제 6 조 의 2"
    """
    if not tok:
        return None
    t = re.sub(r"\s+", "", str(tok))
    m = re.search(r"(?:제)?(\d+)조(?:의(\d+))?", t)
    if not m:
        return None
    main = m.group(1)
    sub = m.group(2)
    return f"제{main}조" if not sub else f"제{main}조의{sub}"

def _law_name_from_doc_meta(meta: dict) -> str:
    # 네 코드에 이미 src_title에서 법령명 추출 로직이 있음(첫 '(' 이전). :contentReference[oaicite:2]{index=2}
    src_title = (meta or {}).get("src_title") or ""
    return _law_name_from_src_title(src_title)

def _split_article_field_local(article_field) -> List[str]:
    # 네 코드에 이미 구분자 split 로직이 있음. :contentReference[oaicite:3]{index=3}
    return _split_article_field(article_field)

def build_allowed_citations_from_docs(docs) -> Set[Tuple[str, str]]:
    """
    retrieved_docs metadata 기반으로 허용 인용 집합 생성:
      (법령명, '제N조/제N조의M')
    """
    allowed: Set[Tuple[str, str]] = set()
    for d in docs:
        meta = getattr(d, "metadata", {}) or {}
        law = _law_name_from_doc_meta(meta)
        for tok in _split_article_field_local(meta.get("article", None)):
            art = _norm_article_token(tok)
            if art:
                allowed.add((law, art))
    return allowed

def extract_answer_citations(answer: str, law_names: List[str]) -> List[Tuple[str, str]]:
    """
    답변에서 (법령명, 조문) 추출.
    law_names는 docs에서 나온 법령명 목록을 넣어 정확도를 높임.
    """
    if not law_names:
        return []
    # 길이 긴 법령명 먼저
    law_names = sorted(set(law_names), key=len, reverse=True)
    law_part = "(" + "|".join(map(re.escape, law_names)) + ")"
    re_law_art = re.compile(rf"(?P<law>{law_part})\s*(?P<art>제\s*\d+\s*조(?:\s*의\s*\d+)?)")
    out = []
    for m in re_law_art.finditer(answer or ""):
        out.append((m.group("law").strip(), _norm_art(m.group("art"))))
    return out

def extract_art_mentions(answer: str) -> List[str]:
    return [_norm_art(m.group(1)) for m in _ART_IN_ANSWER_RE.finditer(answer or "")]

@dataclass
class RuleGateResult:
    completeness: float
    accuracy: float
    n_answer_citations: int
    art_only_sentences: List[str]              # 법령명 없이 조문만 나온 문장들(디버그)
    inaccurate_citations: List[Tuple[str,str]] # (law, art) not in allowed
    debug: Dict

def rule_gate_eval(answer: str, docs) -> RuleGateResult:
    # 1) docs에서 법령명 목록 수집
    law_names = []
    for d in docs:
        meta = getattr(d, "metadata", {}) or {}
        law_names.append(_law_name_from_doc_meta(meta))
    law_names = sorted(set([x for x in law_names if x and x != "알수없음"]), key=len, reverse=True)

    # 2) 허용 인용 집합(메타 기반)
    allowed = build_allowed_citations_from_docs(docs)

    # 3) 답변 인용 추출
    answer_cits = extract_answer_citations(answer, law_names)

    # 4) Accuracy
    if not answer_cits:
        accuracy = 1.0
        inaccurate = []
    else:
        inaccurate = [(law, art) for (law, art) in answer_cits if (law, art) not in allowed]
        accuracy = (len(answer_cits) - len(inaccurate)) / len(answer_cits)

    # 5) Completeness (문장 단위로 엄격하게)
    sentences = [s.strip() for s in _SENT_SPLIT_RE.split(answer or "") if s and s.strip()]
    art_only_sents = []
    total_art_mentions = 0
    ok_art_mentions = 0

    for s in sentences:
        arts = extract_art_mentions(s)
        if not arts:
            continue
        total_art_mentions += len(arts)
        has_law = any(law in s for law in law_names)
        # 문장 안에 법령명이 있으면, 그 문장의 조문 언급은 모두 "완전" 처리
        if has_law:
            ok_art_mentions += len(arts)
        else:
            # 법령명 없는 문장인데 조문 언급이 있음 → completeness 실패 원인
            art_only_sents.append(s)

    completeness = 1.0 if total_art_mentions == 0 else (ok_art_mentions / total_art_mentions)

    return RuleGateResult(
        completeness=completeness,
        accuracy=accuracy,
        n_answer_citations=len(answer_cits),
        art_only_sentences=art_only_sents,
        inaccurate_citations=inaccurate,
        debug={
            "law_names": law_names[:30],
            "allowed_sample": list(sorted(allowed))[:30],
            "answer_citations": answer_cits,
            "total_art_mentions": total_art_mentions,
        }
    )

def pass_rule_gate(rule: RuleGateResult, completeness_min=0.99, accuracy_min=0.90) -> bool:
    if rule.completeness < completeness_min:
        return False
    if rule.accuracy < accuracy_min:
        return False
    return True

# 2. LLM

## index law

In [8]:
# 재사용 가능한 함수로 만들기

def ask_with_reference(query:str, k:int=4, top_k:int=4):
    'query, 표준화된 query, 답변, 참조사항 출력'
    
    # 1. llm, embedding
    
    ### OPENAI ###
    # llm = ChatOpenAI(model='gpt-4o-mini')
    # llm = ChatOpenAI(model='gpt-4.1-mini')
    # llm = ChatOpenAI(model='gpt-4.1-nano')
    # llm = ChatOpenAI(model='gpt-5-mini')
    # llm = ChatOpenAI(model='gpt-5-nano')
    
    ### OLLAMA ###
    # llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    # llm = ChatOllama(model="exaone3.5:7.8b", temperature=0.1)
    # llm = ChatOllama(model="bnksys/yanolja-eeve-korean-instruct-10.8b", temperature=0.1)
    # llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.1)
    # llm = ChatOllama(model="", temperature=0.1)
    
    ### UPSTAGE ###
    llm = ChatUpstage(model = 'solar-pro2')
    
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)

    # 2. 업로드한 벡터 DB에서 가져올 때
    law_database= PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_LAW)

    rule_database = PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_RULE)

    # case_database = PineconeVectorStore(
        # embedding=up_embedding,
        # index_name=PINECONE_INDEX_NAME_CASE)
    
    # 3. keyword_chain 
    keyword_dict = {
        # 1. 계약 주체 및 대상
        "집주인": "임대인", "건물주": "임대인", "주인집": "임대인", "임대업자": "임대인", "새주인": "임대인",
        "세입자": "임차인", "월세입자": "임차인", "세들어사는사람": "임차인", "임차자": "임차인", "입주자": "임차인",
        "부동산": "공인중개사", "중개인": "공인중개사", "중개소": "공인중개사",
        "빌라": "임차주택", "아파트": "임차주택", "오피스텔": "임차주택", "우리집": "임차주택", "거주지": "임차주택",
        "계약서": "임대차계약증서", "집문서": "임대차계약증서", "종이": "임대차계약증서",

        # 2. 보증금 및 금전
        "전세금": "임차보증금", "보증금": "임차보증금", "맡긴돈": "임차보증금", "떼인돈": "임차보증금",
        "월세": "차임", "방세": "차임", "다달이내는지출": "차임", "렌트비": "차임", "임대료": "차임",
        "복비": "중개보수", "수수료": "중개보수", "중개비": "중개보수",
        "월세올리기": "차임증액", "인상": "증액", "더달라고함": "증액", "5프로": "5퍼센트상한",
        "월세깎기": "차임감액", "할인": "감액", "내리기": "감액",
        "돈먼저받기": "우선변제권", "순위": "우선변제권", "안전장치": "대항력", "돌려받기": "보증금반환",
        "보험": "반환보증", "허그": "HUG", "나라보증": "보증보험",

        # 3. 계약 상태 및 변화
        "연장하기": "계약갱신요구권", "한번더살기": "계약갱신", "2플러스2": "계약갱신요구권", "갱신": "계약갱신",
        "재계약": "계약갱신", "자동연장": "묵시적갱신", "연락없음": "묵시적갱신", "그냥연장": "묵시적갱신",
        "이사": "주택의인도", "짐빼기": "주택의인도", "퇴거": "주택의인도", "방빼": "계약해지",
        "주소옮기기": "주민등록", "전입신고": "주민등록", "주소지이전": "주민등록",
        "집주인바뀜": "임대인지위승계", "주인바뀜": "임대인지위승계", "매매": "임대인지위승계",
        "나가라고함": "계약갱신거절", "쫓겨남": "명도", "비워달라": "명도", "중도해지": "계약해지",

        # 4. 수리 및 생활환경
        "집고치기": "수선의무", "수리": "수선의무", "고쳐줘": "수선의무", "안고쳐줌": "수선의무위반",
        "곰팡이": "하자", "물샘": "누수", "보일러고장": "하자", "파손": "훼손",
        "깨끗이치우기": "원상회복의무", "원래대로해놓기": "원상회복", "청소비": "원상회복비용", "청소": "원상회복",
        "층간소음": "공동생활평온", "옆집소음": "방음", "개키우기": "반려동물특약", "담배": "흡연금지특약",

        # 5. 리스크 및 분쟁
        "깡통전세": "전세피해", "사기": "전세사기", "경매넘어감": "권리리스크", "빚": "근저당",
        "세금안냄": "체납", "나라빚": "조세채권", "빌린돈": "가압류", "신탁": "신탁부동산",
        "특약": "특약사항", "불공정": "강행규정위반", "독소조항": "불리한약정", "효력있나": "무효여부",
        "조정위": "주택임대차분쟁조정위원회", "소송말고": "분쟁조정", "법원가기싫음": "분쟁조정",
        "집주인사망": "임차권승계", "자식상속": "임차권승계"
    }
    
    prompt = ChatPromptTemplate.from_template('''아래 질문에서 단어만 사전을 기준으로 치환하세요.문장 구조, 시제, 의문형 여부는 변경하지 마세요
    사전 : {dictionary}
    질문 : {question}
    출력은 질문 문장만 하세요''')

    keyword_chain = prompt | llm | StrOutputParser()
    
    # 4. 질문을 표준화
    def format_keyword_dict(d: dict) -> str:
        return "\n".join([f"{k} → {v}" for k, v in d.items()])
    normalized_query = keyword_chain.invoke({
        "question": query,
        "dictionary": format_keyword_dict(keyword_dict)
    })

    # 5. 관련문서 검색
    # retriever = law_database.as_retriever(search_kwargs = {'k':k, 'filter':{'category':{'$in':categorize_content(query)}}})
    retriever = law_database.as_retriever(search_kwargs={'k': k})

    # 6. 관련문서  rerank
    embed_query = safe_embedding_query(normalized_query, query)
    categories = categorize_content(query)
    
    retrieved_docs = retrieve_law_docs_priority_mix(
        law_db=law_database,
        embedding=up_embedding,
        query_for_embed=embed_query,
        # categories=categories,
        categories=None,
        k_total=max(k, 6),         # 최종 문서 수는 기존 k를 따름
        k_each=8,  # 각 priority에서 몇 개씩 볼지(넉넉히)
        priority_order=[1,2,3,4,5,6,7],
        min_quota={1: max(1, k-1), 2: 1},
    )
    reranked_docs = rerank_by_title(embed_query, retrieved_docs, top_k) # 최종 documents
    referenced_articles = extract_articles_from_docs(reranked_docs) # 참조조항 출력

    # 7. Context 생성
    context = format_documents_with_meta(reranked_docs)

    # 8. 프롬프트
    template = """
    당신은 친절하고 전문적인 주택임대차 전문 변호사입니다.

    ### 답변 작성 규칙:
    1. **질문의 핵심에 먼저 직접 답변**하세요 (예: "네, 나가셔야 합니다" 또는 "아니요, 안 나가셔도 됩니다")
    2. **이유를 쉽고 간결하게** 설명하세요
    3. **근거 법령을 자연스럽게** 언급하세요  
       - 반드시 **법령명 + 조문 번호**를 함께 표기하세요  
       - 예: (주택임대차보호법 제6조의3)
    4. **구체적인 행동 방법**을 단계별로 안내하세요
    5. 친근하고 이해하기 쉬운 말투를 사용하세요
    6. 임차인에게 불리한 내용은 **명확히 강조**하세요

    ### 검색된 법령 및 판례:
    {context}

    ### 질문:
    {question}

    ### 답변 (핵심 답 → 이유 → 근거 → 실행 방법 순서로):
    """
    prompt = ChatPromptTemplate.from_template(template)
    # prompt_chain = prompt | llm | StrOutputParser()
    # result = prompt_chain.invoke({'context':format_documents(retrieved_docs),
                                  # 'query':query})
    prompt_chain = prompt | llm
    llm_response = prompt_chain.invoke({'context':format_documents(reranked_docs),
                                        'question':query})
    result = llm_response.content
    
    # -------------------------
    # RAGAS (기존 유지)
    # -------------------------
    rule = rule_gate_eval(result, reranked_docs)
    rule_pass = pass_rule_gate(rule)
    
    # ======================
    # 비용/요약 출력(기존 유지)
    # ======================
    token_usage = get_token_usage(llm, llm_response)
    cost = calculate_llm_cost("solar-pro2", token_usage)

    def _fmt_money(x):
        try:
            return f"{float(x):.8f}"
        except:
            return str(x)

    def _short(s, n=80):
        s = str(s)
        return (s[:n] + "…") if len(s) > n else s

    ref_preview = _short(referenced_articles, 120)

    fail_reasons = []
    if not rule_pass:
        if rule.completeness < 0.99:
            fail_reasons.append(f"법령명 누락 문장 {len(rule.art_only_sentences)}개")
        if rule.accuracy < 0.90:
            fail_reasons.append(f"컨텍스트 없는 인용 {len(rule.inaccurate_citations)}개")
    fail_reason_text = " / ".join(fail_reasons) if fail_reasons else "-"

    print("\n" + "=" * 90)
    print("🧪 EVAL SUMMARY")
    print("-" * 90)
    print(f"Q(original)   : {_short(query, 90)}")
    print(f"Q(normalized) : {_short(normalized_query, 90)}")
    print(f"Retrieval     : k={k} | top_k={top_k} | pool={len(retrieved_docs)} | final={len(reranked_docs)}")
    print(f"Ref articles  : {ref_preview}")
    print("-" * 90)
    print(f"  - completeness: {rule.completeness:.3f} (min=0.99)")
    print(f"  - accuracy    : {rule.accuracy:.3f} (min=0.90)")
    print(f"  - fail reason : {fail_reason_text}")
    if (not rule_pass) and (rule.completeness < 0.99) and rule.art_only_sentences:
        print(f"  - example(art-only): {_short(rule.art_only_sentences[0], 120)}")
    if (not rule_pass) and (rule.accuracy < 0.90) and rule.inaccurate_citations:
        law, art = rule.inaccurate_citations[0]
        print(f"  - example(bad cite): {law} {art}")
    print("-" * 90)

    if token_usage is None:
        print("Tokens        : None")
    else:
        print(f"Tokens        : in={token_usage['input_tokens']} | out={token_usage['output_tokens']} | total={token_usage['total_tokens']}")
    print(f"Cost(est.)    : {_fmt_money(cost)}")
    print("=" * 90)
    print(result)

    return None


### 답변 비교

In [9]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제3조 ― 대항력 (전입신고·점유) ↔ 시행령 제2조·제4조·제5조·제6조)

ask_with_reference('전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?
Q(normalized) : 전입신고랑 주민등록은 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제2조,제3조,제536조,제575조,제578조 | 부동산 거래신고 등에 관한 법률 시행령 제4조,제6조 | 민법 제631조
------------------------------------------------------------------------------------------
  - completeness: 1.000 (min=0.99)
  - accuracy    : 0.500 (min=0.90)
  - fail reason : 컨텍스트 없는 인용 1개
  - example(bad cite): 주택임대차보호법 제3조의6
------------------------------------------------------------------------------------------
Tokens        : in=1097 | out=441 | total=1538
Cost(est.)    : 0.00042915
### 핵심 답변:  
**"네, 확정일자부 기재 내용을 반드시 확인하셔야 합니다!"**  

### 이유:  
확정일자부에는 **임대차 계약의 핵심 조건(보증금, 월세, 계약 기간 등)**이 기재되며, 이는 향후 분쟁 시 **법적 증거**로 활용됩니다. 특히 임대인이 "계약 내용을 부인하거나 변경하려 할 

In [21]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제4조 ― 임대차기간(2년 보장)

ask_with_reference('계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?
Q(normalized) : 계약갱신요구권에 따라 1년 지나면 무조건 주택의인도를 해야 하는 건가요?
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조 | 민법 제620조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 0.500 (min=0.90)
  - fail reason : 법령명 누락 문장 2개 / 컨텍스트 없는 인용 1개
  - example(art-only): - **동법 제6조의3**: 임차인이 갱신을 요구할 경우, 임대인은 정당한 사유 없이 거절 불가능합니다.
  - example(bad cite): 주택임대차보호법 제8조
------------------------------------------------------------------------------------------
Tokens        : in=1109 | out=314 | total=1423
Cost(est.)    : 0.00035475
**답변:**  
"아니요, 무조건 나가실 필요는 없습니다!"  

### 📌 **이유**  
계약서에 1년으로 작성되었더라도, **주택임대차보호법**에 따라 임차인이 갱신요구를 할 수 있습니다. 단, 임대인이 정당한 사유(예: 직접 거주, 임대차계약 위반 등) 없이 거절할 수 없습니다.

In [11]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제5조 ― 임차인의 전대 제한

ask_with_reference('제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?
Q(normalized) : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 임대인랑 말로만 얘기했어요. 문제 될 수 있나요?
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 부동산 거래신고 등에 관한 법률 시행령 제4조,제6조 | 민법 제629조
------------------------------------------------------------------------------------------
  - completeness: 0.600 (min=0.99)
  - accuracy    : 0.000 (min=0.90)
  - fail reason : 법령명 누락 문장 2개 / 컨텍스트 없는 인용 3개
  - example(art-only): - **동법 제629조 제2항**: 동의 없는 전대 시 임대인은 **계약 해지**를 할 수 있습니다.
  - example(bad cite): 주택임대차보호법 제629조
------------------------------------------------------------------------------------------
Tokens        : in=1168 | out=434 | total=1602
Cost(est.)    : 0.00043560
**네, 문제 될 수 있습니다.**  

### 📌 이유  
집주인의 **사전 서면 동의 없이 친구에게 전대(전세/월세 양도)하는 행위**는 **주택임대차보호법 제629조** 위반입니다. 구두 약속만으로는 법적 

In [12]:
%%time
# 모델 : solar-pro2
# 조항 : 주택임대차보호법 제 6조(계약의 갱신)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
Q(normalized) : 임대인이 계약 끝나기 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 명도하라고 합니다. 그래도 주택의인도를 해야 하나요?
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제6조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조 | 민법 제635조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개
  - example(art-only): - **같은 법 제6조의2 제1항**: 묵시적 갱신 시 임차인은 **언제든지 해지 통지 가능** (단, 해지 효력은 3개월 후 발생)
------------------------------------------------------------------------------------------
Tokens        : in=694 | out=435 | total=1129
Cost(est.)    : 0.00036510
### 핵심 답변:  
**"아니요, 무조건 나가실 필요 없습니다!"**  

### 이유:  
집주인이 계약 만료 1달 전에 갱신 거절 통지를 하지 않았다면, **법은 자동으로 계약이 2년 더 연장된 것으로 봅니다**. 따라서 현재 계약 조건으로 계속 거주하실 권리

In [13]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제6조의3 ― 계약갱신 거절 사유(실거주 등)↔ 시행령 제5조 제5호

ask_with_reference('집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?
Q(normalized) : 임대인이 실거주한다고 계약갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조
------------------------------------------------------------------------------------------
  - completeness: 1.000 (min=0.99)
  - accuracy    : 0.000 (min=0.90)
  - fail reason : 컨텍스트 없는 인용 2개
  - example(bad cite): 주택임대차보호법 제6조의3
------------------------------------------------------------------------------------------
Tokens        : in=1163 | out=420 | total=1583
Cost(est.)    : 0.00042645
**답변:**  
네, 집주인이 실제 거주하지 않는다면 **손해배상을 청구할 수 있습니다**.  

### 📌 이유  
집주인이 "실거주"를 이유로 계약 갱신을 거절했지만 실제로 거주하지 않은 경우, **주택임대차보호법 제6조의3 제5항**에 따라 임차인은 손해배상을 요구할 수 있습니다. 단, **집주인이 제3자에게 재임대한 사실**을 증명해야 합니다.  

### 📜 근거 법령  
- **주택임대차보호

In [14]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조 ― 차임 증감 청구(5% 상한) ↔ 시행령 제8조

ask_with_reference('재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 …
Q(normalized) : 재계약하면서 차임을 한 번에 많이 올리자고 하는데, 임대인이 정하는 대로 따라야 하나요? 그리고 차임을 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 …
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제9조,제15조,제16조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개
  - example(art-only): - **동법 제7조 제2항**: 증액 상한은 5%(1/20)이며, 지자체에서 더 낮출 수 있습니다.
------------------------------------------------------------------------------------------
Tokens        : in=1234 | out=508 | total=1742
Cost(est.)    : 0.00048990
### 핵심 답변:  
**"아니요, 집주인이 정하는 대로 무조건 따를 필요 없습니다. 또, 월세 인상은 1년에 한 번만 가능합니다."**  

---

### 1. 이유 설명:  
- 월세 인상은 **주택임대차

In [15]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조의2 ― 보증금 ↔ 월세 전환 비율 ↔ 시행령 제9조

ask_with_reference('보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?
Q(normalized) : 임차보증금 줄여주는 대신 차임로 바꾸자는데, 차임를 너무 많이 받으려고 해요. 기준 같은 게 있나요?
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제7조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제9조,제15조,제16조
------------------------------------------------------------------------------------------
  - completeness: 1.000 (min=0.99)
  - accuracy    : 0.000 (min=0.90)
  - fail reason : 컨텍스트 없는 인용 3개
  - example(bad cite): 주택임대차보호법 제7조의2
------------------------------------------------------------------------------------------
Tokens        : in=790 | out=583 | total=1373
Cost(est.)    : 0.00046830
**네, 월세 산정 기준이 있습니다!**  
임대인이 보증금을 줄이고 월세로 전환할 때, 월세는 법적으로 정해진 범위를 넘을 수 없습니다. 너무 많이 받으려고 한다면 이의를 제기할 수 있어요.

---

### 📌 **이유와 법적 근거**  
1. **월세 산정률 제한**  
   - 보증금을 월세로 전환할 때는 다음 두 비율 중 **낮은 비율**을 적용해 계산해야 합니다.  
     - **은행 대출금리 + 경제여

In [16]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제8조 ― 보증금 중 일정액 보호(소액임차인) ↔ 시행령 제10조·제11조

ask_with_reference('집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?
Q(normalized) : 경매넘어감 → 권리리스크  
보증금 → 임차보증금  
돈먼저받기 → 우선변제권  
지역마다 → 지역별로  
기준 → 기준  

질문 : 권리리스크로 넘어간다고 들…
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제3조,제4조,제8조,제41조,제152조,제161조,제621조 | 주택임대차보호법 시행령 제3조,제8조,제10조,제17조
------------------------------------------------------------------------------------------
  - completeness: 0.750 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개
  - example(art-only): - **동법 제10조 제1항**: 지역별 최우선 변제 한도 명시
------------------------------------------------------------------------------------------
Tokens        : in=1685 | out=418 | total=2103
Cost(est.)    : 0.00050355
**네, 보증금 중 일정액을 최우선으로 받을 수 있습니다!**  

### 1. ▶ **이유**  
주택임대차보호법 제8조와 제10조에 따라, 임차인은 경매 시 보증금 중 **"일정액"을 다른 채권자보다 먼저 돌려받을 

#### 특정시행령 안가져오지만 부동산중개업법으로 보완

In [17]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제14조 제1·2항(주택임대차분쟁조정위원회 + 대상 분쟁) ↔ 시행령 제22조

ask_with_reference('집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.
Q(normalized) : 임대인랑 차임랑 임차보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 임대인이 공인중개사 중개보수를 저보고 다 내라고 하는데요.
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조,제8조,제14조,제20조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조
------------------------------------------------------------------------------------------
  - completeness: 0.667 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개
  - example(art-only): - 중개사는 원칙적으로 **임대인·임차인 절반씩 부담**해야 합니다(「부동산 중개업법」 제23조).
------------------------------------------------------------------------------------------
Tokens        : in=1400 | out=392 | total=1792
Cost(est.)    : 0.00044520
**네, 주택임대차분쟁조정위원회에서 조정을 받을 수 있습니다.**  
(단, 중개사 수수료 문제는 조정 대상이 아닙니다. 별도 소송이 필요할 수 있어요.)

### ✔️ **이유**  
주택임대차분쟁조정위원회는 월세,

#### 시행령 안가져옴

In [18]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제21조 제1항(조정 신청 관할) ↔ 시행령 제30조·제33조

ask_with_reference('제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈…
Q(normalized) : 제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 임차주택에서 하면 되나요, 원래 임차주택 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 …
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제14조,제21조,제24조 | 주택임대차보호법 시행령 제3조,제4조,제8조,제10조,제11조,제17조,제18조
------------------------------------------------------------------------------------------
  - completeness: 0.750 (min=0.99)
  - accuracy    : 0.667 (min=0.90)
  - fail reason : 법령명 누락 문장 1개 / 컨텍스트 없는 인용 1개
  - example(art-only): *(근거: 제21조 제3항 제5호)*
  - example(bad cite): 주택임대차보호법 시행령 제21조
------------------------------------------------------------------------------------------
Tokens        : in=1420 | out=493 | total=1913
Cost(est.)    : 0.00050880
**답변:**  
네, **지금 사는 곳이 아닌 원래 임대차계약이 체결된 주택 소재지 관할 조정위원회에 신청**해야 합니다. (주택임대차보호법 제21

#### 시행령 안가져옴

In [19]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제22조(조정절차 개시 및 송달) ↔ 시행령 제32조

ask_with_reference('조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?
Q(normalized) : 조정 신청하고 나면 임대인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제14조,제21조,제24조 | 주택임대차보호법 시행령 제2조,제3조,제7조,제8조,제10조,제15조,제17조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개
  - example(art-only): - **제24조**: 조정위원회는 필요한 경우 직접 조사하거나 시·도지사에 자료를 요청할 수 있어, 임대인의 협조 없이도 절차를 진행할 수 있습니다.
------------------------------------------------------------------------------------------
Tokens        : in=1215 | out=347 | total=1562
Cost(est.)    : 0.00039045
### 핵심 답변:  
**"아니요, 집주인에게 바로 연락가지 않습니다. 조정 신청은 비공개로 진행되지만, 임대인이 답변하지 않아도 자동으로 불리해지지는 않습니다."**  

---

### 이유 설명:  
1. **비공개 원

#### 확실한거 안가져옴

In [20]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제27조(조정서의 집행력) ↔ 시행령 제34·35조

ask_with_reference('조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.')


🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.
Q(normalized) : 조정에서 합의했는데 임대인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 임대차계약증서인지 판결 같은 건지 헷갈려요.
Retrieval     : k=4 | top_k=4 | pool=6 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조,제8조,제14조,제20조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조
------------------------------------------------------------------------------------------
  - completeness: 1.000 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : -
------------------------------------------------------------------------------------------
Tokens        : in=1397 | out=384 | total=1781
Cost(est.)    : 0.00043995
**답변:**  
**"아니요, 조정결과는 법적 효력이 있어 강제 집행이 가능합니다!"**  

### 1. **이유**  
주택임대차분쟁조정위원회에서 합의한 내용은 **법원 판결과 동일한 효력**을 가집니다. 집주인이 합의를 지키지 않으면 **강제 집행**을 통해 보증금·차임 등을 받을 수 있습니다.  

### 2. **근거 법령**  
- **주택임대차보호법 제14조 제5항**에 따라 조정서는 **확정판